# Rを利用した細胞診 (FNA) 結果からの乳がん診断

## 概要

このノートブックでは、細胞診結果にもとづく乳がん診断をRを使って行います。細胞の画像から直接診断するのではなく、細胞の均一さなどの特徴から診断します。このような特徴量と診断結果のデータセットが、[Breast Cancer Wisconsin (Diagnostic) Data Set](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic))として公開されているので、これを利用します。


## データのダウンロードと確認

まずデータをダウンロードします。ダウンロードする `breast-cancer-wisconsin.data`はヘッダのない csv 形式のファイルです。`read.csv`すると、適当に設定されたヘッダ名とともにデータを見ることができます。


In [ ]:
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data"
data <- read.csv(url)
head(data)

このままだとデータがわかりにくいので正しいヘッダを付与します。
- id: サンプルのID
- CT: 細胞の塊の厚さ
- UCSize: 細胞サイズの均一さ
- UCShape: 細胞の形の均一さ
- MA: 辺縁接着
- SECS: 単一上皮細胞サイズ
- BN: 裸核
- BC: 無刺激性クロマチン
- NN: 通常の核
- M: 分裂
- diagnosis: 良性なら2、悪性なら4

この問題は良性の2と悪性の4を認識する問題ですが、2種類に分ける際、通常は0か1に分けることが多いです。そこで、 outcome という列を作って、diagnosis が 4 なら outcome を 1 に、diagnosis が 2 なら outcome を 0 にするようにします。

In [ ]:
data <- read.csv(file = url, header = FALSE,
                col.names = c("id","CT", "UCSize", "UCShape", "MA", "SECS", "BN", "BC", "NN","M", "diagnosis") )

data$outcome[data$diagnosis==4] = 1
data$outcome[data$diagnosis==2] = 0
data$outcome = as.integer(data$outcome)
head(data)

## データの前処理と分析

データのうち id は予測と無関係な値であることが自明ですので除去しましょう。また、BNには欠損値が含まれていますので、今回は削除します。

In [ ]:
library(dplyr)
library(tidyverse)
data2 <- data %>% select(-id, -BN)

data2$outcome[data2$diagnosis==4] = 1
data2$outcome[data2$diagnosis==2] = 0
data2$outcome = as.integer(data2$outcome)
head(data2)

PerformanceAnalytics のライブラリをインストールすることでデータ間の相関を分析することができます。

In [ ]:
install.packages("PerformanceAnalytics")
library(PerformanceAnalytics)
chart.Correlation(data2[, c(1:10)], histogram=TRUE, col="grey10", pch=1, main="Cancer Means")

データセットで良性と悪性の分布をグラフ化します。

In [ ]:
ggplot(data2, aes(x = outcome)) +
  geom_bar(aes(fill = "blue")) +
  ggtitle("Distribution of diagnosis for the entire dataset") +
  theme(legend.position="none")

データの8割を学習データにして、残りをテストデータにします。学習データのみを学習に利用して、テストデータでモデルを評価します。

In [ ]:
sample_size = floor(0.8 * nrow(data2))

set.seed(0)
train_set = sample(seq_len(nrow(data2)), size = sample_size)

training = data2[train_set, ]

testing = data2[-train_set, ]

head(training)

## モデルの学習

良性か悪性かの2つに分類する問題のため、単純な方式であるロジスティック回帰を利用します。ロジスティック回帰とは、以下のような回帰式です。

$P=\frac{e^y}{1+e^y}$  
$y = w_0 + w_1 x_1 + w_2 x_2 + \cdots w_n x_n$

ここで CT や UCSize の値が $x_1$や $x_2$になります。ただし、yを outcomeそのものとして線形回帰を行うと、0や1以外の値を取りうる可能性が出てくるので、シグモイド関数を取り入れて 0から1の間になるようにします。シグモイド関数を計算して得られる P は、例えば、良性や悪性の確率とみなすことができます。

R でロジスティック回帰を行う際は glm を利用します。これは一般化線形モデルのライブラリで、2項分布 binomial を仮定すればロジスティック回帰になります。どの説明変数を使うか、どれを目的変数とするかは、数式のような形で指定します。

In [ ]:
model_algorithm = model = glm(outcome ~ CT + 
                                UCSize +
                                UCShape +
                                MA +
                                SECS + 
                                BC  +
                                NN  +
                                M ,
                               family=binomial(link='logit'), control = list(maxit = 50),data=training)

print(summary(model_algorithm))

学習が終わったらテストしてみましょう。悪性の確率が0.5を超えるものは悪性とみなします。

In [ ]:
prediction_testing = predict(model_algorithm,testing, type = "response")
prediction_testing = ifelse(prediction_testing > 0.5, 1, 0)
error = mean(prediction_testing != testing$outcome)
print(paste('Model Accuracy',1-error))

上記では0.5を悪性か良性かの判断に使いましたが、0.5よりも小さくすると多くを悪性と見なす傾向になり、悪性の見落としは減りますが、良性も悪性とみなすため診断効率は下がります。良い診断というのは、どのようなしきい値でも良い精度が得られるというものです。

これを評価するために、しきい値を変えながら悪性の正しい判定（True Positive）と良性の誤検出 (False Positive) を計算し、カーブをかいたものがROC曲線とよばれているものです。カーブの右上はしきい値が最も低い状況で、悪性も全て検出しますが（True Positive =1.0)、良性も悪性と検出します（False Positive =1.0)。

このROC曲線の内側の領域 (AUC: Area under curve) が広いほうが良い評価となります。

In [ ]:
install.packages("ROCR")
library(ROCR)
p = predict(model_algorithm, testing, type="response")
pr = prediction(p, testing$outcome)
prf = performance(pr, measure = "tpr", x.measure = "fpr")
plot(prf)

## SageMaker を利用して学習と推論を行う

より大規模かつ複雑なデータの場合に、ロジスティック回帰よりも高度なアルゴリズムを使いたくなるかもしれません。SageMakerでは、表形式のデータに対して高い分類性能を誇るXGBoostを利用できます。詳細は以下の例も参考にしてください。

https://github.com/aws/amazon-sagemaker-examples/blob/master/r_examples/r_batch_transform/r_xgboost_batch_transform.ipynb

まずは必要なライブラリをインストールします。SageMaker のPythonパッケージを使うために、reticulate をインストールしてからSageMakerをインポートします。

In [ ]:
install.packages('tictoc', repos='http://cran.us.r-project.org')
library(tictoc)
# Install reticulate library and import sagemaker
library(reticulate)
sagemaker <- import('sagemaker')

SageMaker に必要なセッションの情報や保存先 (bucket)などを指定します。

In [ ]:
session <- sagemaker$Session()
bucket <- session$default_bucket()
prefix <- 'r-batch-transform'
role_arn <- sagemaker$get_execution_role()

さきほどロジスティック回帰で利用したデータをここでも利用します。**注意点**として、SageMakerのXGBoostを利用する際は、**一列目にラベルが必要**ですので列の順番を入れ替えます。

In [ ]:
xgb_data <- data2
xgb_data <- xgb_data[, c(10, 1, 2, 3, 4, 5, 6, 7, 8)]
head(xgb_data)

学習データとバリデーションデータを用意します。用意したら SageMaker の学習用に CSV ファイルとして保存します。

In [ ]:
sample_size = floor(0.8 * nrow(xgb_data))

set.seed(0)
train_valid_set = sample(seq_len(nrow(xgb_data)), size = sample_size)
train_valid_data = xgb_data[train_valid_set, ]
testing = xgb_data[-train_valid_set, ]

sample_size = floor(0.8 * nrow(train_valid_data))
train_set = sample(seq_len(nrow(train_valid_data)), size = sample_size)
training = train_valid_data[train_set, ]
validation = train_valid_data[-train_set, ]

write_csv(training, 'train.csv', col_names = FALSE)
write_csv(validation, 'valid.csv', col_names = FALSE)

SageMaker の upload_data の関数でそれぞれのファイルを S3 にアップロードします。

In [ ]:
s3_train <- session$upload_data(path = 'train.csv', 
                                bucket = bucket, 
                                key_prefix = 'data')
s3_valid <- session$upload_data(path = 'valid.csv', 
                                bucket = bucket, 
                                key_prefix = 'data')
s3_train_input <- sagemaker$inputs$TrainingInput(s3_data = s3_train,
                                     content_type = 'csv')
s3_valid_input <- sagemaker$inputs$TrainingInput(s3_data = s3_valid,
                                     content_type = 'csv')

SageMaker のXGBoost のアルゴリズムは、AWS が管理するコンテナイメージで提供されます。そのイメージの場所を知っておく必要があるため、get_image_uriで取得します。


In [ ]:
container <- sagemaker$amazon$amazon_estimator$get_image_uri(session$boto_region_name, 'xgboost',repo_version='1.0-1')
container

取得が完了すれば、そのコンテナイメージに加えて、学習用のインスタンスタイプや数を指定して学習を行います。

In [ ]:
s3_output <- paste0('s3://', bucket, '/output')
estimator <- sagemaker$estimator$Estimator(image_uri = container,
                                           role = role_arn,
                                           train_instance_count = 1L,
                                           train_instance_type = 'ml.m5.large')


tic("Model Fitting")
estimator$set_hyperparameters(num_round = 100L,objective = 'binary:logistic')
job_name <- paste('sagemaker-train-xgboost', format(Sys.time(), '%H-%M-%S'), sep = '-')
input_data <- list('train' = s3_train_input,
                   'validation' = s3_valid_input)
estimator$fit(inputs = input_data, job_name = job_name)
toc()

学習が終わるとモデルをホストして、推論を行う環境を構築することができます。CSV形式でデータを送信するので、CSVSerializer を指定します。

In [ ]:
tic("Model Deployment")
model_endpoint <- estimator$deploy(initial_instance_count = 1L,
                                   instance_type = 'ml.t2.medium')
model_endpoint$serializer <- sagemaker$predictor$CSVSerializer()
toc()

テストデータを取り出してエンドポイントにリクエストを送って結果を取得します。

In [ ]:
test_data<- testing[-1]
num_predict_rows <- nrow(testing)
test_sample <- as.matrix(test_data[1:num_predict_rows, ])
dimnames(test_sample)[[2]] <- NULL


In [ ]:
tic("Invoke Endpoint")
library(stringr)
predictions <- model_endpoint$predict(test_sample)
predictions <- str_split(predictions, pattern = ',', simplify = TRUE)
predictions <- as.numeric(predictions)
predictions = ifelse(predictions > 0.5, 1, 0)
toc()

取得した結果を表示したり、精度・ROCを確認します。

In [ ]:
# Convert predictions to Integer
result <- cbind(predicted_outcome = as.integer(predictions), 
                      testing[1:num_predict_rows, ])
head(result)

In [ ]:
error = mean(result$predicted_outcome != result$outcome)
print(paste('Model Accuracy',1-error))

In [ ]:
pr = prediction(result$predicted_outcome, result$outcome)
prf = performance(pr, measure = "tpr", x.measure = "fpr")
plot(prf)

最後にエンドポイントを削除します。

In [ ]:
session$delete_endpoint(model_endpoint$endpoint)